# <img src="https://img.icons8.com/external-flaticons-flat-flat-icons/150/000000/external-research-university-flaticons-flat-flat-icons-2.png" width="120" style="vertical-align:middle; margin-right:10px;" />
# <span style="font-size:1.3em; vertical-align:middle;"><b>Deep Research:</b> Unlocking cross-business insights with agentic intelligence</span>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
                <svg width="150" height="150" viewBox="0 0 150 150" style="display: block;">
                    <defs>
                        <radialGradient id="grad1" cx="50%" cy="50%" r="50%" fx="50%" fy="50%">
                            <stop offset="0%" style="stop-color:#00bfff;stop-opacity:1" />
                            <stop offset="100%" style="stop-color:#005fa3;stop-opacity:1" />
                        </radialGradient>
                    </defs>
                    <circle cx="75" cy="75" r="70" fill="url(#grad1)" stroke="#222" stroke-width="3"/>
                    <g>
                        <rect x="35" y="60" width="80" height="40" rx="12" fill="#fff" stroke="#00bfff" stroke-width="3"/>
                        <rect x="50" y="45" width="50" height="20" rx="6" fill="#e6f7ff" stroke="#00bfff" stroke-width="2"/>
                        <rect x="60" y="105" width="30" height="10" rx="3" fill="#e6f7ff" stroke="#00bfff" stroke-width="2"/>
                    </g>
                    <text x="75" y="90" text-anchor="middle" font-size="22" font-family="Verdana" fill="#00bfff" font-weight="bold" letter-spacing="2">DR</text>
                    <text x="75" y="130" text-anchor="middle" font-size="14" font-family="Verdana" fill="#fff" font-weight="bold">Deep Research</text>
                </svg>
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # This is to ensure the agent uses the tools - Mandatory for tool to be used
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems:

- **LangGraph**: An extension of LangChain, LangGraph introduces a graph-based system for managing agent workflows, supporting complex decision-making processes with features like branching, looping, and asynchronous task execution. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen focuses on multi-agent systems, enabling agents to communicate in natural language to collaboratively complete tasks. It includes built-in agents and a graphical UI for prototyping, facilitating the creation of complex agent interactions. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **CrewAI**: Designed for orchestrating role-based AI agents, CrewAI emphasizes simplicity and minimal setup, making it popular for applications in customer service and marketing automation. It allows for the definition of agent roles and collaborative workflows with minimal code. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, the OpenAI Agents SDK focuses on creating multi-agent workflows with comprehensive tracing and guardrails, facilitating the development of complex agent interactions. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **Eliza**: A Web3-friendly AI agent operating system, Eliza integrates seamlessly with blockchain applications, allowing AI agents to interact with smart contracts and blockchain data, expanding the scope of decentralized applications. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

- **AutoAgent**: A fully automated, zero-code framework for LLM agents, AutoAgent enables users to create and deploy AI agents through natural language, making AI agent development accessible to non-technical users. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and approaches to meet various application needs. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [ ]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To gather information on the most recent AI agent frameworks released in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To find articles and reports detailing innovations and advancements in AI agents for the year 2025.', query='AI agent technology trends 2025'), WebSearchItem(reason='To explore comparisons and reviews of AI frameworks available for developers in 2025.', query='best AI frameworks for agents 2025')]


In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("adnanmdashikml@gmail.com") # Change this to your verified email
    to_email = To("adnanmdashikml@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109278cc0>, strict_json_schema=True, is_enabled=True)

In [9]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [10]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [11]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [13]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces